In [69]:
from helpers import *
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error



In [70]:
work_dfs = {}
for file in glob("./cultivos/*.csv"):
    df = pd.read_csv(file)
    filename= Path(file).stem
    work_dfs[filename] = df
    
work_dfs.keys()

dict_keys(['Barley', 'Oilseeds_total', 'Rice', 'Sweet_potato', 'Tapioca'])

## Preprocesamiento y modelado

Ahora se ejecuta un pipeline que aplica `StandardScaler` y un modelo (`Ridge`, `RandomForest`,`MLPRegressor`) con sus respectivas grillas de hiperparámetros para comparar resultados. Los resultados se almacenan en `regressors` con el nombre del cultivo como clave y el modelo como valor

In [71]:
regressors = {} 
for name, df in work_dfs.items():
    grouped = df.set_index("Crop_Year")

    X_train, X_test, y_train, y_test = train_test_split(grouped.drop("Yield", axis=1), grouped["Yield"], test_size=0.2, random_state=42)

    # Definir los modelos en un diccionario con sus parámetros
    pipelines = {
        'Ridge': Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())]),
        'RandomForest': Pipeline([('scaler', StandardScaler()), ('regressor', RandomForestRegressor(random_state=42))]),
        'MLP': Pipeline([('scaler', StandardScaler()), ('regressor', MLPRegressor(max_iter=5000, random_state=42))])
    }

    param_grids = {
        'Ridge': {'regressor__alpha': [0.01, 0.1, 1.0, 10.0]},
        'RandomForest': {'regressor__n_estimators': [50, 100,500], 'regressor__max_depth': [None, 5]},
        'MLP': {'regressor__hidden_layer_sizes': [(50,), (100,)], 'regressor__alpha': [0.0001, 0.001,0.01,0.1]}
    }

    # Ejecutar GridSearch para cada modelo
    mejores_resultados = {}

    for nombre, pipeline in pipelines.items():
        grid = GridSearchCV(pipeline, param_grids[nombre], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        grid.fit(X_train, y_train)
        y_pred = grid.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mejores_resultados[nombre] = {'BestParams': grid.best_params_, 'RMSE': rmse, 'Model': grid.best_estimator_}
        

    # Mostrar resultados
    for modelo, resultado in mejores_resultados.items():
        print(f"Para {name}")
        print(f"Modelo: {modelo}")
        print(f"Mejores Parámetros: {resultado['BestParams']}")
        print(f"RMSE: {resultado['RMSE']:.2f}")
        print("-" * 30)
    best_model = min(mejores_resultados, key=lambda k: mejores_resultados[k]['RMSE'])
    regressors[name] =  mejores_resultados.get(best_model).get("Model")


Para Barley
Modelo: Ridge
Mejores Parámetros: {'regressor__alpha': 10.0}
RMSE: 0.76
------------------------------
Para Barley
Modelo: RandomForest
Mejores Parámetros: {'regressor__max_depth': None, 'regressor__n_estimators': 50}
RMSE: 0.64
------------------------------
Para Barley
Modelo: MLP
Mejores Parámetros: {'regressor__alpha': 0.0001, 'regressor__hidden_layer_sizes': (100,)}
RMSE: 0.63
------------------------------
Para Oilseeds_total
Modelo: Ridge
Mejores Parámetros: {'regressor__alpha': 1.0}
RMSE: 0.05
------------------------------
Para Oilseeds_total
Modelo: RandomForest
Mejores Parámetros: {'regressor__max_depth': None, 'regressor__n_estimators': 100}
RMSE: 0.06
------------------------------
Para Oilseeds_total
Modelo: MLP
Mejores Parámetros: {'regressor__alpha': 0.1, 'regressor__hidden_layer_sizes': (50,)}
RMSE: 0.13
------------------------------
Para Rice
Modelo: Ridge
Mejores Parámetros: {'regressor__alpha': 1.0}
RMSE: 0.66
------------------------------
Para Rice
Mo

### Resultados

In [72]:
resultados = pd.DataFrame.from_dict(regressors, columns=["Mejor modelo"],orient="index")
resultados["Mejor modelo"] = resultados["Mejor modelo"].apply(lambda x: x.steps[1][1])
resultados

,Mejor modelo
Barley,"MLPRegressor(max_iter=5000, random_state=42)"
Oilseeds_total,Ridge()
Rice,"(DecisionTreeRegressor(max_depth=5, max_featur..."
Sweet_potato,"(DecisionTreeRegressor(max_depth=5, max_featur..."
Tapioca,"(DecisionTreeRegressor(max_features=1.0, rando..."


Se puede hacer una prediccion para el cultivo elegido con su mejor modelo al seleccionar la clave y pasar un nuevo vector de variables. Por completitud, se pide un dataframe con los nombres de las columnas. Pero funciona también sin los headers

### Algunas predicciones

In [73]:
sample = pd.DataFrame([{"Annual_Rainfall":940.700000,"Fertilizer":18014.620000,"Pesticide":356.700000}])
regressors.get("Barley").predict(sample)


array([1.0199958])

In [74]:
regressors.get("Sweet_potato").predict(sample)

array([0.904978])